In [ ]:
import numpy as np
from scipy.linalg import expm

# Quaternion utils

In [3]:
# Helper: Quaternion multiplication
def quaternion_multiply(q1, q2):
    """
    Multiply two quaternions.
    """
    w1, x1, y1, z1 = q1
    w2, x2, y2, z2 = q2
    return np.array([
        w1 * w2 - x1 * x2 - y1 * y2 - z1 * z2,
        w1 * x2 + x1 * w2 + y1 * z2 - z1 * y2,
        w1 * y2 - x1 * z2 + y1 * w2 + z1 * x2,
        w1 * z2 + x1 * y2 - y1 * x2 + z1 * w2
    ])


def quaternion_derivative(q, w):
    """
    Compute the quaternion derivative.

    Parameters:
    - q: Current state quaternion [w, x, y, z]
    - w: Angular velocity vector [wx, wy, wz]

    Returns:
    - q_dot: Quaternion derivative
    """
    qw, qx, qy, qz = q
    wx, wy, wz = w

    q_dot = 0.5 * np.array([
        -qx * wx - qy * wy - qz * wz,
         qw * wx + qy * wz - qz * wy,
         qw * wy - qx * wz + qz * wx,
         qw * wz + qx * wy - qy * wx
    ])

    return q_dot

def f(q, w, dt):
    """
    State prediction function.

    Parameters:
    - q: Current state quaternion [w, x, y, z]
    - w: Angular velocity vector [wx, wy, wz]
    - dt: Time step

    Returns:
    - q_hat: Predicted state quaternion
    """
    q_dot = quaternion_derivative(q, w)

    # Euler integration
    q_hat = q + dt * q_dot
    q_hat /= np.linalg.norm(q_hat)  # Normalize quaternion

    return q_hat

# Helper: Normalize quaternion
def normalize_quaternion(q):
    """
    Ensure the quaternion is normalized (unit norm).
    """
    return q / np.linalg.norm(q)

# Helper: Skew-symmetric matrix for cross products
def skew_symmetric(v):
    return np.array([
        [0, -v[2], v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0]
    ])

def quaternion_derivative_matrix(omega):
    """
    Calculate the quaternion derivative matrix.
    """
    return np.array([
        [0, -omega[0], -omega[1], -omega[2]],
        [omega[0], 0, omega[2], -omega[1]],
        [omega[1], -omega[2], 0, omega[0]],
        [omega[2], omega[1], -omega[0], 0]
    ])


# Predykcja

## Stan 

\begin{split}
\begin{array}{rcl}
\hat{\mathbf{q}}_t &=& \mathbf{f}(\mathbf{q}_{t-1}, \mathbf{\omega}_t) \\
\hat{\mathbf{q}}_t &=& \mathbf{q}_{t-1} + \int_{t-1}^t\boldsymbol\omega\, dt
\end{array}
\end{split}

In [4]:
def f(q, w, dt):
    """
    State prediction function.

    Parameters:
    - q: Current state quaternion [w, x, y, z]
    - w: Angular velocity vector [wx, wy, wz]
    - dt: Time step

    Returns:
    - q_hat: Predicted state quaternion
    """
    # Quaternion derivative
    q_dot = quaternion_derivative(q, w)

    # Euler integration
    q_hat = q + dt * q_dot
    q_hat /= np.linalg.norm(q_hat)  # Normalize quaternion

    return q_hat

## Kowariancja predykcji stanu

Z definicji 

\begin{split}\begin{array}{rcl}
\hat{\mathbf{P}}_t &=& \mathbf{F}(\mathbf{q}_{t-1}, \mathbf{\omega}_t)\mathbf{P}_{t-1}\mathbf{F}^T(\mathbf{q}_{t-1}, \mathbf{\omega}_t) + \mathbf{Q}_t
\end{array}\end{split}


### Pochodna predykcji stanu

Przyblizenie pierwszego rzedu z Taylora.


\begin{split}\begin{array}{rcl}
\mathbf{F}(\mathbf{q}_{t-1}, \boldsymbol\omega_t)
&=& \frac{\partial\mathbf{f}(\mathbf{q}_{t-1}, \boldsymbol\omega_t)}{\partial\mathbf{q}} \\
&=&\begin{bmatrix}
\frac{\partial\mathbf{f}(\mathbf{q}_{t-1}, \boldsymbol\omega_t)}{\partial q_w} &
\frac{\partial\mathbf{f}(\mathbf{q}_{t-1}, \boldsymbol\omega_t)}{\partial q_x} &
\frac{\partial\mathbf{f}(\mathbf{q}_{t-1}, \boldsymbol\omega_t)}{\partial q_y} &
\frac{\partial\mathbf{f}(\mathbf{q}_{t-1}, \boldsymbol\omega_t)}{\partial q_z}
\end{bmatrix} \\ &=&
\begin{bmatrix}
    1 & - \frac{\Delta t}{2} \omega_x & - \frac{\Delta t}{2} \omega_y & - \frac{\Delta t}{2} \omega_z\\
    \frac{\Delta t}{2} \omega_x & 1 & \frac{\Delta t}{2} \omega_z & - \frac{\Delta t}{2} \omega_y\\
    \frac{\Delta t}{2} \omega_y & - \frac{\Delta t}{2} \omega_z & 1 & \frac{\Delta t}{2} \omega_x\\
    \frac{\Delta t}{2} \omega_z & \frac{\Delta t}{2} \omega_y & - \frac{\Delta t}{2} \omega_x & 1
\end{bmatrix}
\end{array}\end{split}

In [ ]:
def F_jacobian(w, dt):
    """
    Compute the Jacobian of the state prediction function f with respect to the state quaternion.

    Parameters:
    - w: Angular velocity vector [wx, wy, wz]
    - dt: Time step

    Returns:
    - F: 4x4 Jacobian matrix
    """
    wx, wy, wz = w
    dq = dt / 2.0

    F = np.array([
        [1,     -dq * wx, -dq * wy, -dq * wz],
        [dq * wx,    1,  dq * wz, -dq * wy],
        [dq * wy, -dq * wz,    1,  dq * wx],
        [dq * wz,  dq * wy, -dq * wx,    1]
    ])

    return F

### Szum procesu

Szum pochodzi glownie z zyroskopu. Gęstość szumu przyjmujemy
\begin{bmatrix}\sigma_{\omega x}^2 & \sigma_{\omega y}^2 & \sigma_{\omega z}^2\end{bmatrix}^T

\begin{split}\Sigma_\omega =
\begin{bmatrix}
\sigma_{\omega x}^2 & 0 & 0 \\
0 & \sigma_{\omega y}^2 & 0 \\
0 & 0 & \sigma_{\omega z}^2
\end{bmatrix}\end{split}

Calkujemy wplyw szumu w jednostce czasu 

\begin{split}
\mathbf{Q}_t = \int_0^{\Delta t} e^{\mathbf{A}_t} \Sigma_\omega e^{\mathbf{A}_t^T} dt
\end{split}

Powyzszy wzór przyblizamy Taylorem 

\begin{split}\begin{array}{rcl}
\mathbf{W}_t &=& \frac{\partial\mathbf{f}(\mathbf{q}_{t-1}, \boldsymbol\omega_t)}{\partial\boldsymbol\omega} \\
&=& \begin{bmatrix}
\frac{\partial\mathbf{f}(\mathbf{q}_{t-1}, \boldsymbol\omega_t)}{\partial\omega_x} &
\frac{\partial\mathbf{f}(\mathbf{q}_{t-1}, \boldsymbol\omega_t)}{\partial\omega_y} &
\frac{\partial\mathbf{f}(\mathbf{q}_{t-1}, \boldsymbol\omega_t)}{\partial\omega_z}
\end{bmatrix} \\
&=& \frac{\Delta t}{2}
\begin{bmatrix}
-q_x & -q_y & -q_z \\
q_w & -q_z & q_y \\
q_z & q_w & -q_x \\
-q_y & q_x & q_w
\end{bmatrix}
\end{array}\end{split}

In [ ]:
def Q_process_noise(q, gyroNoise, dt):
    """
    Compute the process noise covariance matrix Q.

    Parameters:
    - q: Current state quaternion [w, x, y, z]
    - gyroNoise: 3-element array of gyroscope noise variances
    - dt: Time step

    Returns:
    - Q: 4x4 process noise covariance matrix
    """
    qw, qx, qy, qz = q
    wx_noise, wy_noise, wz_noise = gyroNoise

    E = np.diag([wx_noise, wy_noise, wz_noise])  # 3x3 noise covariance

    # Jacobian of the process model with respect to the noise
    W = (dt / 2.0) * np.array([
        [-qx, -qy, -qz],
        [ qw, -qz,  qy],
        [ qz,  qw, -qx],
        [-qy,  qx,  qw]
    ])  # Shape: (4,3)

    Q = W @ E @ W.T  # Shape: (4,4)

    return Q


### Kowariancja procesu

Wzór jw.

In [ ]:
def P_hat_prediction(P, q, w, gyroNoise, dt):
    """
    Predict the next state covariance matrix P_hat.

    Parameters:
    - P: Current state covariance matrix (4x4)
    - q: Current state quaternion [w, x, y, z]
    - w: Angular velocity vector [wx, wy, wz]
    - gyroNoise: 3-element array of gyroscope noise variances
    - dt: Time step

    Returns:
    - P_hat: Predicted state covariance matrix (4x4)
    """
    F = F_jacobian(w, dt)
    Q = Q_process_noise(q, gyroNoise, dt)
    P_hat = F @ P @ F.T + Q
    return P_hat

# Korekcja

\begin{split}
\mathbf{q}_t = \hat{\mathbf{q}}_t + \mathbf{K}_t \big(\mathbf{z}_t - \mathbf{h}(\mathbf{q}_t)\big)
\end{split}

## Spodziewane wskazania akcelerometru

\begin{split}\begin{array}{rcl}
\mathbf{h}(\hat{\mathbf{q}}_t) &=& \hat{\mathbf{a}}
= \mathbf{C}(\hat{\mathbf{q}})^T \mathbf{g} \\
&=&
2 \begin{bmatrix}
g_x (\frac{1}{2} - q_y^2 - q_z^2) + g_y (q_wq_z + q_xq_y) + g_z (q_xq_z - q_wq_y) \\
g_x (q_xq_y - q_wq_z) + g_y (\frac{1}{2} - q_x^2 -  q_z^2) + g_z (q_wq_x + q_yq_z) \\
g_x (q_wq_y + q_xq_z) + g_y (q_yq_z - q_wq_x) + g_z (\frac{1}{2} - q_x^2 -  q_y^2)
\end{bmatrix}
\end{array}\end{split}


gdzie $\mathbf{C}(\mathbf{q})\in\mathbb{R}^{3\times 3}$ jest macierzą rotacji kwaternionu

$ g = [0, 0, 1] $

In [ ]:
def h(q, g):
    """
    Measurement function mapping the state quaternion to expected accelerometer measurements.

    Parameters:
    - q: State quaternion [w, x, y, z]
    - g: Gravity vector in ENU frame [gx, gy, gz]

    Returns:
    - h: 3-element measurement prediction vector
    """
    qw, qx, qy, qz = q
    gx, gy, gz = g

    # Accelerometer measurement prediction (gravity)
    acc_pred = 2 * np.array([
        gx * (0.5 - qy**2 - qz**2) +
        gy * (qw * qz + qx * qy) +
        gz * (qx * qz - qw * qy),

        gx * (qx * qy - qw * qz) +
        gy * (0.5 - qx**2 - qz**2) +
        gz * (qw * qx + qy * qz),

        gx * (qw * qy + qx * qz) +
        gy * (qy * qz - qw * qx) +
        gz * (0.5 - qx**2 - qy**2)
    ])

    return acc_pred

## Kalman gain

\begin{split}\begin{array}{rcl}
\mathbf{K}_t &=& \hat{\mathbf{P}}_t \mathbf{H}^T(\hat{\mathbf{q}}_t) \mathbf{S}_t^{-1}
\end{array}\end{split}

### Pochodna spodziewanych wskazań akcelerometru

znowu z jakobianu

\begin{split}\begin{array}{rcl}
\mathbf{H}(\hat{\mathbf{q}}_t) &=& \frac{\partial\mathbf{h}(\hat{\mathbf{q}}_t)}{\partial\mathbf{q}} \\
&=&\begin{bmatrix}
\frac{\partial\mathbf{h}(\hat{\mathbf{q}}_t)}{\partial q_w} &
\frac{\partial\mathbf{h}(\hat{\mathbf{q}}_t)}{\partial q_x} &
\frac{\partial\mathbf{h}(\hat{\mathbf{q}}_t)}{\partial q_y} &
\frac{\partial\mathbf{h}(\hat{\mathbf{q}}_t)}{\partial q_z}
\end{bmatrix} \\
&=&
2
\begin{bmatrix}
g_xq_w + g_yq_z - g_zq_y & g_xq_x + g_yq_y + g_zq_z & -g_xq_y + g_yq_x - g_zq_w & -g_xq_z + g_yq_w + g_zq_x \\
-g_xq_z + g_yq_w + g_zq_x & g_xq_y - g_yq_x + g_zq_w &  g_xq_x + g_yq_y + g_zq_z & -g_xq_w - g_yq_z + g_zq_y \\
g_xq_y - g_yq_x + g_zq_w & g_xq_z - g_yq_w - g_zq_x &  g_xq_w + g_yq_z - g_zq_y &  g_xq_x + g_yq_y + g_zq_z
\end{bmatrix}
\end{array}\end{split}


In [ ]:
def H_jacobian(q, g):
    """
    Compute the Jacobian matrix H of the measurement function h with respect to the state quaternion.

    Parameters:
    - q: State quaternion [w, x, y, z]
    - g: Gravity vector in ENU frame [gx, gy, gz]

    Returns:
    - H: 3x4 Jacobian matrix
    """
    qw, qx, qy, qz = q
    gx, gy, gz = g

    # Jacobian with respect to accelerometer prediction only
    H = np.zeros((3, 4))

    H[0, :] = 2 * np.array([
        gy * qz - gz * qy,
        gx * (-2 * qy) + gy * qw + gz * qz,
        -2 * gx * qy + gy * qz - gz * qw,
        -2 * gx * qz - gy * qy + gz * qx
    ])

    H[1, :] = 2 * np.array([
        gy * qw + gz * qx,
        gx * qy + gy * qx + gz * qz,
        gx * qx - gy * qw + gz * qz,
        gx * qz - gy * qx - gz * qw
    ])

    H[2, :] = 2 * np.array([
        -gx * qy + gy * qx,
        gx * qz + gy * qy - gz * qw,
        gx * qw - gy * qz + gz * qx,
        gx * qx + gy * qy - gz * qw
    ])

    return H


### Kowariancja akcelerometru + ostateczny wynik

\begin{split}\begin{array}{rcl}
\mathbf{S}_t &=& \mathbf{H}(\hat{\mathbf{q}}_t) \hat{\mathbf{P}}_t \mathbf{H}^T(\hat{\mathbf{q}}_t) + \mathbf{R} \\
\mathbf{K}_t &=& \hat{\mathbf{P}}_t \mathbf{H}^T(\hat{\mathbf{q}}_t) \mathbf{S}_t^{-1}
\end{array}\end{split}

In [ ]:
def calculate_kalman_gain(q_hat, P_hat, z_t, accNoise, g):
    """
    Calculate the Kalman gain K.

    Parameters:
    - q_hat: Current state quaternion [w, x, y, z]
    - P_hat: Predicted state covariance matrix (4x4)
    - z_t: Measurement vector
    - accNoise: Measurement noise covariance (3-element array)
    - g: Gravity vector in ENU frame [gx, gy, gz]

    Returns:
    - K: Kalman gain matrix
    - q_hat: Updated state quaternion
    """
    # Predict accelerometer measurement
    h_q_hat = h(q_hat, g)
    
    # Jacobian of measurement function
    H = H_jacobian(q_hat, g)
    
    # Measurement noise covariance
    R = np.diag(accNoise)
    
    # Residual covariance
    S = H @ P_hat @ H.T + R
    
    # Kalman gain
    K = P_hat @ H.T @ np.linalg.inv(S)

    return K

## Aktualizacja zmiennych

\begin{split}\begin{array}{rcl}
\mathbf{v}_t &=& \mathbf{z}_t - \mathbf{h}(\hat{\mathbf{q}}_t) \\
\mathbf{q}_t &=& \hat{\mathbf{q}}_t + \mathbf{K}_t \mathbf{v}_t \\
\mathbf{P}_t &=& \big(\mathbf{I}_4 - \mathbf{K}_t\mathbf{H}(\hat{\mathbf{q}}_t)\big)\hat{\mathbf{P}}_t
\end{array}\end{split}

In [ ]:
# Update quaternion estimate with measurement residual
v_t = z_t - h_q_hat  # Measurement residual
q_hat = q_hat + K @ v_t  # Update state estimate
q_hat /= np.linalg.norm(q_hat)  # Re-normalize quaternion

# Update covariance
self.P[:, :, t] = (np.eye(4) - K @ H) @ P_hat

# Save the orientation estimate
self.orientation[t] = q_hat